## Purpose
This notebook computes the DynusT network wide VMT, VHT, and VHD using 15 min. link level outputs produced by Analysis Tool 2. 


In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import os

%matplotlib inline

In [2]:
def to_long_form(df_measure_file, summary_interval, simulation_horizon, measure, number_attr=5):
    """
    df_measure_file: the raw measures dataframe read from csv
    number_attr: number of attributes, the default is [Link,Anode,Bnode,Dist,Lanes]
    summary_interval: the length of simulation interval in minutes
    simulation_horizon: the length of simulation horizon in minutes
    measure: the name of the measure
    """
    
    number_intervals = int(simulation_horizon / summary_interval)
    columns = df_measure_file.columns.tolist()
    intervals_in_file = columns[number_attr:]
    
    if len(intervals_in_file) != number_intervals:
        raise Exception("The specified number of intervals {:d} != number of intervals {:d} in the input data".format(number_intervals,len(intervals_in_file)))
    
    id_vars = df_measure_file.columns.tolist()[:5]
    value_vars = df_measure_file.columns.tolist()[5:]
    df_clean = pd.melt(df_measure_file, id_vars=id_vars, value_vars=value_vars, var_name='Interval', value_name=measure)
    
    # cast interval to integer
    df_clean['Interval'] = df_clean.Interval.astype(int)
    
    # add string time stamps
    stamps = pd.date_range('1/1/2011', periods=number_intervals, freq='15Min').strftime('%H:%M:%S').values
    stamps_dict = {i:stamp for i, stamp in zip(range(1, number_intervals+1), stamps)}
    df_clean['TimeStamp'] = df_clean.Interval.map(stamps_dict)
    return df_clean

In [3]:
def compare_profiles(dfs, nodes, names, title, simulation_interval, measure_name):
    if len(dfs) != len(nodes) or len(dfs) != len(names) or len(nodes) != len(names):
        raise Exception('Data inconsistency data = {:d}, nodes = {:d}, names = {:d}'.format((len(dfs), len(nodes), len(names))))
    
    colors = 'gbmr'                                                                                        
    
    measures = []
    end_interval = int(24 * 60 / simulation_interval)
    selection_columns = ['Link', 'Interval', 'TimeStamp'] + [measure_name]
    for i, (df, node, name) in enumerate(zip(dfs, nodes, names)):                                                                                   
        anode, bnode = node
        measure = df.loc[(df.Anode == anode) & (df.Bnode == bnode) & (df.Interval <= end_interval), selection_columns]
        print('Number of observations for Measure {:d} = {:d}'.format(i, len(measure)))
        x, y, xlabels = measure.Interval.values, measure[measure_name].values, measure.TimeStamp.values
        measures.append((x, y, xlabels, name))
    
    
    xticks = np.arange(0, end_interval, int(60/simulation_interval))
    xticklabels = measures[0][2][xticks]
    
    fig, ax = plt.subplots(figsize=(16,8));
    for i, (measure, color) in enumerate(zip(measures, colors)):                                                                                    
        ax.plot(measure[0], measure[1], color+'-', label=measure[3]);
    
    ax.set_ylabel(measure_name);
    ax.set_xticks(ticks=xticks)
    ax.set_xticklabels(xticklabels, rotation='vertical')
    ax.set_title(title)
    ax.legend()
    
    return ax

### Compute VMT, VHT

In [9]:
DYNUST_DATA_FOLDER = r'Q:\Houston_8_County_DTA_CalibValid\_rpt'
DynusT_travel_time_file = 'OutLinkTravelTime_HGAC_17_full.csv'
DynusT_flow_file = 'OutFlow_HGAC_17_full.csv'
DynusT_travel_time_file = os.path.join(DYNUST_DATA_FOLDER, DynusT_travel_time_file)
DynusT_flow_file = os.path.join(DYNUST_DATA_FOLDER, DynusT_flow_file)
DynusT_travel_time_file, DynusT_flow_file

('Q:\\Houston_8_County_DTA_CalibValid\\_rpt\\OutLinkTravelTime_HGAC_17_full.csv',
 'Q:\\Houston_8_County_DTA_CalibValid\\_rpt\\OutFlow_HGAC_17_full.csv')

In [10]:
simulation_horizon, simulation_interval = 1620, 15
number_intervals = int(simulation_horizon / simulation_interval)
measure_name_travel_time, measure_name_flow = 'TravelTime', 'Flow'

In [18]:
df_travel_time = pd.read_csv(DynusT_travel_time_file)
df_travel_time = df_travel_time.drop(df_travel_time.columns[-1], axis=1)
df_travel_time = to_long_form(df_travel_time, summary_interval=simulation_interval, simulation_horizon=simulation_horizon, measure=measure_name_travel_time)
df_travel_time = df_travel_time.loc[df_travel_time.Interval < 97, :]
df_travel_time.head()

,Link,Anode,Bnode,Dist,Lanes,Interval,TravelTime,TimeStamp
0,1,6001,6063,0.161,2,1,0.24,00:00:00
1,2,6001,6080,0.855,2,1,0.68,00:00:00
2,3,6002,6069,0.021,2,1,0.05,00:00:00
3,4,6002,6074,2.865,2,1,2.87,00:00:00
4,5,6003,419660,1.110,2,1,1.21,00:00:00


In [20]:
df_flow = pd.read_csv(DynusT_flow_file)
df_flow = df_flow.drop(df_flow.columns[-1], axis=1)
df_flow = to_long_form(df_flow, summary_interval=simulation_interval, simulation_horizon=simulation_horizon, measure=measure_name_flow)
df_flow = df_flow.loc[df_flow.Interval < 97, :]
df_flow.head()

,Link,Anode,Bnode,Dist,Lanes,Interval,Flow,TimeStamp
0,1,6001,6063,0.161,2,1,0.0,00:00:00
1,2,6001,6080,0.855,2,1,1.0,00:00:00
2,3,6002,6069,0.021,2,1,18.0,00:00:00
3,4,6002,6074,2.865,2,1,0.0,00:00:00
4,5,6003,419660,1.110,2,1,0.0,00:00:00


In [22]:
join_columns = ['Link', 'Interval', 'Anode', 'Bnode', 'TimeStamp', 'Lanes', 'Dist']
df_link_moe = pd.merge(left=df_travel_time, right=df_flow, left_on=join_columns, right_on=join_columns, how='inner')

In [23]:
df_link_moe.head()

,Link,Anode,Bnode,Dist,Lanes,Interval,TravelTime,TimeStamp,Flow
0,1,6001,6063,0.161,2,1,0.24,00:00:00,0.0
1,2,6001,6080,0.855,2,1,0.68,00:00:00,1.0
2,3,6002,6069,0.021,2,1,0.05,00:00:00,18.0
3,4,6002,6074,2.865,2,1,2.87,00:00:00,0.0
4,5,6003,419660,1.110,2,1,1.21,00:00:00,0.0


### Compute Delay

In [29]:
TRANSIMS_network_folder = r'Q:\TRANSIMS\Network'
TRANSIMS_network_file = os.path.join(TRANSIMS_network_folder, r'Link.csv')
TRANSIMS_network_file

'Q:\\TRANSIMS\\Network\\Link.csv'

In [30]:
df_link = pd.read_csv(TRANSIMS_network_file)
df_link.head()

,LINK,NAME,NODE_A,NODE_B,LENGTH,SETBACK_A,SETBACK_B,BEARING_A,BEARING_B,TYPE,...,LANES_AB,SPEED_AB,FSPD_AB,CAP_AB,LANES_BA,SPEED_BA,FSPD_BA,CAP_BA,USE,NOTES
0,1,NaN,6001,6063,852.0,0.0,0.0,0,0,RAMP,...,2,55.0,55.0,3316,0,0.0,0.0,0,ANY,NaN
1,2,NaN,6001,6080,4512.0,0.0,0.0,0,0,FREEWAY,...,2,75.0,75.0,5666,0,0.0,0.0,0,ANY,NaN
2,3,NaN,6002,6069,110.0,0.0,0.0,0,0,EXPRESSWAY,...,2,60.0,60.0,3252,0,0.0,0.0,0,ANY,NaN
3,4,NaN,6002,6074,15128.0,0.0,0.0,0,0,EXPRESSWAY,...,2,60.0,60.0,3252,0,0.0,0.0,0,ANY,NaN
4,5,NaN,6003,419660,5858.0,0.0,0.0,0,0,EXPRESSWAY,...,2,55.0,55.0,2980,0,0.0,0.0,0,ANY,NaN


In [32]:
df_link['FF_TIME'] = df_link['LENGTH'] / 5280.0 / df_link['FSPD_AB'] * 3600

In [33]:
df_link_FF_TIME = df_link.loc[:, ['NODE_A', 'NODE_B', 'FF_TIME']]

In [34]:
df_link_moe = pd.merge(left=df_link_moe, right=df_link_FF_TIME, left_on=['Anode', 'Bnode'], right_on=['NODE_A', 'NODE_B'], how='inner') 

In [36]:
df_link_moe['DELAY'] = np.where(df_link_moe.TravelTime * 60 < df_link_moe.FF_TIME, 0.0, df_link_moe.TravelTime * 60 - df_link_moe.FF_TIME)

In [37]:
total_VMT = np.dot(df_link_moe.Dist.values, df_link_moe.Flow.values)
total_VHT = np.dot(df_link_moe.TravelTime.values, df_link_moe.Flow.values) / 60.0
avg_speed = total_VMT / total_VHT
total_VHD = np.dot(df_link_moe.DELAY.values, df_link_moe.Flow.values) / 3600
print('Total VMT           = {:,.0f}'.format(total_VMT))
print('Total VHT           = {:,.0f}'.format(total_VHT))
print('Total VHD           = {:,.0f}'.format(total_VHD))
print('Average Speed (mph) = {:,.1f}'.format(avg_speed))

Total VMT           = 191,161,660
Total VHT           = 5,066,471
Total VHD           = 1,396,348
Average Speed (mph) = 37.7
